## Install Glove

In [1]:
!git clone git@github.com:stanfordnlp/GloVe.git
%cd gloVe
!make
%cd ..

fatal: destination path 'GloVe' already exists and is not an empty directory.
/Users/davidlassner/research/conferences_and_papers/workshops/2020 DHd/website/material/notebooks/GloVe
mkdir -p build
/Users/davidlassner/research/conferences_and_papers/workshops/2020 DHd/website/material/notebooks


## Download the aggressive wikipedia comments data set

In [2]:
!mkdir data
!wget https://ndownloader.figshare.com/files/7394506 -O agression_annotation.tsv
!wget https://ndownloader.figshare.com/files/7038038 -O aggression_annotated_comments.tsv
!mv agression_annotation.tsv data/
!mv aggression_annotated_comments.tsv data/

mkdir: data: File exists
--2020-01-31 16:01:37--  https://ndownloader.figshare.com/files/7394506
Resolving ndownloader.figshare.com (ndownloader.figshare.com)... 52.19.35.46, 34.248.199.177, 108.128.77.47, ...
Connecting to ndownloader.figshare.com (ndownloader.figshare.com)|52.19.35.46|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://s3-eu-west-1.amazonaws.com/pfigshare-u-files/7394506/aggression_annotations.tsv [following]
--2020-01-31 16:01:37--  https://s3-eu-west-1.amazonaws.com/pfigshare-u-files/7394506/aggression_annotations.tsv
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.100.179
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.100.179|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30627328 (29M) [binary/octet-stream]
Saving to: ‘agression_annotation.tsv’

agression_annotatio 100%[===================>]  29.21M  43.2MB/s    in 0.7s    

2020-01-31 16:01:38 

## Preprocess data

In [1]:
import pandas as pd
from tqdm.notebook import tqdm
aggression_annotated_comments = pd.read_csv("data/aggression_annotated_comments.tsv", sep="\t")
agression_annotation = pd.read_csv("data/agression_annotation.tsv", sep="\t")

agression_data = pd.merge(aggression_annotated_comments, agression_annotation, on="rev_id")

In [2]:
comments = []
labels = []

def regex_filter(comment):
    return comment.replace("NEWLINE_TOKEN", "\t")

for rev_id, rev in tqdm(agression_data.groupby("rev_id")):
    comments.append(regex_filter(rev.iloc[0].comment))
    labels.append(rev.aggression.sum()/len(rev) >.5)

In [71]:
from sklearn.model_selection import train_test_split    
train_embeddings, X_other, embeddings_labels, y_other  = train_test_split(comments, labels, random_state=123)
X_train, X_test, y_train, y_test = train_test_split(X_other, y_other, random_state=456)

In [4]:
from spacy.lang.en import English
nlp = English()
tokenizer = nlp.Defaults.create_tokenizer(nlp)

In [7]:
with open("data/all_comments_plain.txt", "w") as all_fout:
    with open("data/aggressive_comments_plain.txt", "w") as agg_fout:
        with open("data/non_aggressive_comments_plain.txt", "w") as non_agg_fout:
            for comment, label in tqdm(
                zip(tokenizer.pipe(train_embeddings), embeddings_labels),
                total=len(train_embeddings),
                desc="storing comments"
            ):
                out_line = " ".join([t.text.replace("\n", "\t") for t in comment])
                all_fout.write("{}\n".format(out_line))
                if label == 1:
                    agg_fout.write("{}\n".format(out_line))
                else:
                    non_agg_fout.write("{}\n".format(out_line))

KeyboardInterrupt: 

In [7]:
!ls data

agg_vec_data.tsv
agg_vec_meta.tsv
aggression_annotated_comments.tsv
aggressive_comments_cooccurrences.bin
aggressive_comments_cooccurrences_shuffled.bin
aggressive_comments_plain.txt
aggressive_comments_vec.txt
agression_annotation.tsv
all_comments_cooccurrences.bin
all_comments_cooccurrences_shuffled.bin
all_comments_plain.txt
all_comments_vec.txt
all_vec_data.tsv
all_vec_meta.tsv
noagg_vec_data.tsv
noagg_vec_meta.tsv
non_aggressive_comments_cooccurrences.bin
non_aggressive_comments_cooccurrences_shuffled.bin
non_aggressive_comments_plain.txt
non_aggressive_comments_vec.txt
vocab.txt


In [8]:
!ls gloVe/build/

common.o      cooccur.o     glove.o       shuffle.o     vocab_count.o
cooccur       glove         shuffle       vocab_count


## Build Vocabulary

In [9]:
!gloVe/build/vocab_count -max-vocab 25000 -min-count 10 < data/all_comments_plain.txt > data/vocab.txt 

BUILDING VOCABULARY
Processed 0 tokens.100000 tokens.200000 tokens.300000 tokens.400000 tokens.500000 tokens.600000 tokens.700000 tokens.800000 tokens.900000 tokens.1000000 tokens.1100000 tokens.1200000 tokens.1300000 tokens.1400000 tokens.1500000 tokens.1600000 tokens.1700000 tokens.1800000 tokens.1900000 tokens.2000000 tokens.2100000 tokens.2200000 tokens.2300000 tokens.2400000 tokens.2500000 tokens.2600000 tokens.2700000 tokens.2800000 tokens.2900000 tokens.3000000 tokens.3100000 tokens.3200000 tokens.3300000 tokens.3400000 tokens.3500000 tokens.3600000 tokens.3700000 tokens.3800000 tokens.3900000 tokens.4000000 tokens.4100000 tokens.4200000 tokens.4300000 tokens.4400000 tokens.4500000 tokens.4600000 tokens.4700000 tokens.4800000 tokens.4900000 tokens.5000000 tokens.5100000 tokens.5200000 tokens.5300000 tokens.5400000 tokens.5500000 tokens.5600000 tokens.5700000 tokens.5800000 tokens.5900000 tokens.6000000 tokens.6100000 tokens.6200000 tokens.6300000 tokens.6400000 tokens.6500000 to

## Create cooccurrences

In [10]:
!gloVe/build/cooccur -vocab-file data/vocab.txt < data/non_aggressive_comments_plain.txt > data/non_aggressive_comments_cooccurrences.bin
!gloVe/build/cooccur -vocab-file data/vocab.txt < data/aggressive_comments_plain.txt > data/aggressive_comments_cooccurrences.bin
!gloVe/build/cooccur -vocab-file data/vocab.txt < data/all_comments_plain.txt > data/all_comments_cooccurrences.bin

COUNTING COOCCURRENCES
window size: 15
context: symmetric
max product: 10485784
overflow length: 28521267
Reading vocab from file "data/vocab.txt"...loaded 22657 words.
Building lookup table...table contains 51262847 elements.
Processing token: 0100000200000300000400000500000600000700000800000900000100000011000001200000130000014000001500000160000017000001800000190000020000002100000220000023000002400000250000026000002700000280000029000003000000310000032000003300000340000035000003600000370000038000003900000400000041000004200000430000044000004500000460000047000004800000490000050000005100000520000053000005400000550000056000005700000580000059000006000000610000062000006300000640000065000006600000Processed 6668532 tokens.
Writing cooccurrences to disk........2 files in total.
Merging cooccurrence files: processed 0 lines.100000 lines.200000 lines.300000 lines.400000 lines.500000 lines.600000 lines.700000 lines.800000 lines.900000 lines.1000000 lines.1100000 lines.1200000 lines.1300000 lines.1

In [11]:
!gloVe/build/shuffle -verbose 2 -memory 4 < data/non_aggressive_comments_cooccurrences.bin > data/non_aggressive_comments_cooccurrences_shuffled.bin
!gloVe/build/shuffle -verbose 2 -memory 4 < data/aggressive_comments_cooccurrences.bin > data/aggressive_comments_cooccurrences_shuffled.bin
!gloVe/build/shuffle -verbose 2 -memory 4 < data/all_comments_cooccurrences.bin > data/all_comments_cooccurrences_shuffled.bin

Using random seed 1580483054
SHUFFLING COOCCURRENCES
array size: 255013683
Shuffling by chunks: processed 0 lines.processed 15671104 lines.
Wrote 1 temporary file(s).
Merging temp files: processed 0 lines.15671104 lines.Merging temp files: processed 15671104 lines.

Using random seed 1580483061
SHUFFLING COOCCURRENCES
array size: 255013683
Shuffling by chunks: processed 0 lines.processed 2843613 lines.
Wrote 1 temporary file(s).
Merging temp files: processed 0 lines.2843613 lines.Merging temp files: processed 2843613 lines.

Using random seed 1580483062
SHUFFLING COOCCURRENCES
array size: 255013683
Shuffling by chunks: processed 0 lines.processed 16595758 lines.
Wrote 1 temporary file(s).
Merging temp files: processed 0 lines.16595758 lines.Merging temp files: processed 16595758 lines.



## Train Glove

In [12]:
!gloVe/build/glove -input-file data/non_aggressive_comments_cooccurrences_shuffled.bin -vocab-file data/vocab.txt -save-file data/non_aggressive_comments_vec -verbose 2 -vector-size 32 -threads 8 -alpha 0.75 -x-max 100.0 -eta 0.05 -binary 0 -model 2
!gloVe/build/glove -input-file data/aggressive_comments_cooccurrences_shuffled.bin -vocab-file data/vocab.txt -save-file data/aggressive_comments_vec -verbose 2 -vector-size 32 -threads 8 -alpha 0.75 -x-max 100.0 -eta 0.05 -binary 0 -model 2
!gloVe/build/glove -input-file data/all_comments_cooccurrences_shuffled.bin -vocab-file data/vocab.txt -save-file data/all_comments_vec -verbose 2 -vector-size 32 -threads 8 -alpha 0.75 -x-max 100.0 -eta 0.05 -binary 0 -model 2

TRAINING MODEL
Read 15671104 lines.
Initializing parameters...Using random seed 1580483070
done.
vector size: 32
vocab size: 22657
x_max: 100.000000
alpha: 0.750000
01/31/20 - 04:04.33PM, iter: 001, cost: 0.022223
01/31/20 - 04:04.34PM, iter: 002, cost: 0.014752
01/31/20 - 04:04.36PM, iter: 003, cost: 0.013140
01/31/20 - 04:04.37PM, iter: 004, cost: 0.011990
01/31/20 - 04:04.39PM, iter: 005, cost: 0.011016
01/31/20 - 04:04.41PM, iter: 006, cost: 0.010252
01/31/20 - 04:04.42PM, iter: 007, cost: 0.009696
01/31/20 - 04:04.44PM, iter: 008, cost: 0.009289
01/31/20 - 04:04.45PM, iter: 009, cost: 0.008974
01/31/20 - 04:04.47PM, iter: 010, cost: 0.008727
01/31/20 - 04:04.49PM, iter: 011, cost: 0.008527
01/31/20 - 04:04.50PM, iter: 012, cost: 0.008361
01/31/20 - 04:04.52PM, iter: 013, cost: 0.008222
01/31/20 - 04:04.53PM, iter: 014, cost: 0.008103
01/31/20 - 04:04.55PM, iter: 015, cost: 0.008001
01/31/20 - 04:04.57PM, iter: 016, cost: 0.007908
01/31/20 - 04:04.59PM, iter: 017, cost: 0.007828
01

## Load Embeddings

In [5]:
from bisect import bisect_left
import numpy as np
import os

with open("data/vocab.txt") as fin:
    vocab,_ = zip(*map(lambda x: x.split(" "), fin))
vocab = sorted(list(vocab) + ["<unk>"])

def save_for_tf_projector(embedding, vocab, outdir, identifier):
    out_path_data = os.path.join(outdir, "{}_data.tsv".format(identifier))
    out_path_meta = os.path.join(outdir, "{}_meta.tsv".format(identifier))

    with open(out_path_data, "w") as fout:
        for row in embedding:
            fout.write("{}\n".format("\t".join(map(str, row.tolist()))))
    
    with open(out_path_meta, "w") as fout:
        for word in vocab:
            fout.write("{}\n".format(word.encode("utf-8")))

In [6]:
def index(a, x):
    'Locate the leftmost value exactly equal to x'
    i = bisect_left(a, x)
    if i != len(a) and a[i] == x:
        return i
    raise ValueError

def load_embedding(path_, vocab):
    dim = 32
    mat = np.empty((len(vocab), dim))
    
    with open(path_) as fin:
        for row in tqdm(fin):
            splitted = row.replace("\n", "").split(" ")
            key, vec = splitted[0], splitted[1:]
            mat[index(vocab, key)] = vec
            
    return mat
            
all_vec = load_embedding("data/all_comments_vec.txt", vocab)
aggressive_vec = load_embedding("data/aggressive_comments_vec.txt", vocab)
non_aggressive_vec = load_embedding("data/non_aggressive_comments_vec.txt", vocab)

save_for_tf_projector(all_vec, vocab, "data", "all_vec")
save_for_tf_projector(aggressive_vec, vocab, "data", "agg_vec")
save_for_tf_projector(non_aggressive_vec, vocab, "data", "noagg_vec")

In [15]:
len(all_vec)

22658

In [16]:
#from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.metrics import classification_report

def embed_X(X, vocab, embedding, verbose=True):
    embedded_X = np.zeros((len(X), embedding.shape[1]))
    num_misses = 0
    num_hits = 0
    for icomment, comment in tqdm(enumerate(tokenizer.pipe(X)), total=len(X)):
        for token in comment:
            try:
                embedded_X[icomment] += embedding[index(vocab, token.text.replace("\n", "\t"))]
                num_hits += 1
            except ValueError:
                num_misses += 1
                if verbose:
                    print("{} not in vocab".format(token))
        embedded_X[icomment] /= len(comment)
    
    print("misses/hits {}".format(num_misses/num_hits))
    return embedded_X

def train(X,y):
      
    #clf = KNeighborsClassifier()
    clf = svm.SVC()
    
    clf.fit(X, y)
    return clf

def evaluate(clf, X, y_true):
    y_pred = clf.predict(X)
    print(classification_report(y_true, y_pred))

print("all")

clf = train(embed_X(X_train, vocab, all_vec, verbose=False), y_train)
evaluate(clf, embed_X(X_test, vocab, all_vec, verbose=False), y_test)

print("aggressive_vec")

clf = train(embed_X(X_train, vocab, aggressive_vec, verbose=False), y_train)
evaluate(clf, embed_X(X_test, vocab, aggressive_vec, verbose=False), y_test)

print("non_aggressive_vec")

clf = train(embed_X(X_train, vocab, non_aggressive_vec, verbose=False), y_train)
evaluate(clf, embed_X(X_test, vocab, non_aggressive_vec, verbose=False), y_test)

all



misses/hits 0.09561145453507156



misses/hits 0.08990938594837083
              precision    recall  f1-score   support

       False       0.91      0.99      0.95      6292
        True       0.85      0.37      0.51       950

    accuracy                           0.91      7242
   macro avg       0.88      0.68      0.73      7242
weighted avg       0.90      0.91      0.89      7242

aggressive_vec



misses/hits 0.09561145453507156



misses/hits 0.08990938594837083
              precision    recall  f1-score   support

       False       0.90      0.99      0.95      6292
        True       0.84      0.30      0.44       950

    accuracy                           0.90      7242
   macro avg       0.87      0.64      0.69      7242
weighted avg       0.89      0.90      0.88      7242

non_aggressive_vec



misses/hits 0.09561145453507156



misses/hits 0.08990938594837083
              precision    recall  f1-score   support

       False       0.89      0.99      0.94      6292
        True       0.75      0.22      0.34       950

    accuracy                           0.89      7242
   macro avg       0.82      0.60      0.64      7242
weighted avg       0.87      0.89      0.86      7242



In [72]:
import torch
from torch import nn 
import torch.nn.functional as F

def encode_X(X, vocab, verbose=True, max_len=120):
    encoded_X = np.zeros((len(X), max_len))
    num_misses = 0
    num_hits = 0
    for icomment, comment in tqdm(enumerate(tokenizer.pipe(X)), total=len(X)):
        for itoken, token in enumerate(comment):
            if itoken <max_len:
                try:
                    encoded_X[icomment, itoken] = index(vocab, token.text.replace("\n", "\t"))
                    num_hits += 1
                except ValueError:
                    num_misses += 1
                    if verbose:
                        print("{} not in vocab".format(token))
    
    print("misses/hits {}".format(num_misses/num_hits))
    return encoded_X

encoded_X_train = torch.from_numpy(encode_X(X_train, vocab, verbose=False)).long()#[:1000]
encoded_X_test = torch.from_numpy(encode_X(X_test, vocab, verbose=False)).long()#[:1000]
#y_train = y_train
#y_test = y_test


misses/hits 0.09848288826812693



misses/hits 0.09583071954045633


In [83]:
from sklearn.metrics import accuracy_score
def it_batches(X,y, batch_size=16):
    
    permutation = torch.randperm(X.size()[0])
    y = torch.tensor(y, dtype=torch.long)

    for i in range(0,X.size()[0], batch_size):
        indices = permutation[i:i+batch_size]
        yield X[indices], y[indices]


        
class WordAttNet(nn.Module):
    """
    @author: Viet Nguyen <nhviet1009@gmail.com>
    """
    def __init__(self, embedding_matrix, hidden_size=50):
        super(WordAttNet, self).__init__()
        
            
        vocab_size, embed_size = embedding_matrix.shape
        
        self.word_weight = nn.Parameter(torch.Tensor(2 * hidden_size, 2 * hidden_size))
        self.word_bias = nn.Parameter(torch.Tensor(1, 2 * hidden_size))
        self.context_weight = nn.Parameter(torch.Tensor(2 * hidden_size, 1))
        
        self.lookup = nn.Embedding(*embedding_matrix.shape)
        self.lookup.weight = nn.Parameter(torch.from_numpy(embedding_matrix))
        self.lookup.weight.requires_grad = False
    
        self.gru = nn.GRU(embed_size, hidden_size, bidirectional=True)
        self._create_weights(mean=0.0, std=0.05)
        self.fc = nn.Linear(2 * hidden_size, 2)

    def _create_weights(self, mean=0.0, std=0.05):

        self.word_weight.data.normal_(mean, std)
        self.context_weight.data.normal_(mean, std)

    def forward(self, input, hidden_state):
        output = self.lookup(input)
        output = output.permute(1,0,2)

        f_output, h_output = self.gru(output.float(), hidden_state)  # feature output and hidden state output
        output = matrix_mul(f_output, self.word_weight, self.word_bias)
        output = matrix_mul(output, self.context_weight).permute(1,0)

        output = F.softmax(output)
        output = element_wise_mul(f_output,output.permute(1,0))
        output = self.fc(output)

        return output, h_output

batch_size = 16
hidden_size = 32
model = WordAttNet(embedding_matrix=all_vec, hidden_size=hidden_size)

def element_wise_mul(input1, input2):

    feature_list = []
    for feature_1, feature_2 in zip(input1, input2):
        feature_2 = feature_2.unsqueeze(1).expand_as(feature_1)
        feature = feature_1 * feature_2
        feature_list.append(feature.unsqueeze(0))
    output = torch.cat(feature_list, 0)
    return torch.sum(output, 0).unsqueeze(0)
    
def matrix_mul(input, weight, bias=False):
    feature_list = []
    for feature in input:
        feature = torch.mm(feature, weight)
        if isinstance(bias, torch.nn.parameter.Parameter):
            feature = feature + bias.expand(feature.size()[0], bias.size()[1])
        feature = torch.tanh(feature).unsqueeze(0)
        feature_list.append(feature)

    return torch.cat(feature_list, 0).squeeze()


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

epochs = 10

train_loss_hist = []
test_loss_hist = []

for epoch in range(epochs):

    pbar = tqdm(it_batches(encoded_X_train, y_train, batch_size=batch_size), total=len(encoded_X_train)//batch_size)
    
    print("### EPOCH {} ###".format(epoch))
    model.train()
    c_loss = 0
    n_batches = 0
    for X_batch, y_batch in pbar:
        if len(X_batch) == batch_size:
            optimizer.zero_grad()

            hidden_state = torch.zeros(2, batch_size, hidden_size)

            output,_ = model(X_batch, hidden_state)
            loss = criterion(output.squeeze(0), y_batch)
            y_pred = np.argmax(output.detach(), -1)
            loss.backward()
            optimizer.step()

            c_loss += loss.item()
            n_batches += 1
        
        pbar.set_description("train loss {0:.4f}".format(c_loss/n_batches))
        
    train_loss_hist.append(c_loss/n_batches)
    
    model.eval()
    c_loss = 0
    n_batches = 0
    accuracy = 0
    for X_batch, y_batch in tqdm(it_batches(encoded_X_test, y_test, batch_size=batch_size), desc="eval", total=len(encoded_X_test)//batch_size):
        if len(X_batch) == batch_size:
            optimizer.zero_grad()
            with torch.no_grad():
                hidden_state = torch.zeros(2, batch_size, hidden_size)
                output,_ = model(X_batch, hidden_state)
                y_pred = np.argmax(output.detach(), -1)
                
                accuracy += accuracy_score(y_batch.numpy().ravel(), y_pred.numpy().ravel())
            loss = criterion(output.squeeze(0), y_batch)

            c_loss += loss.item()
            n_batches += 1

    test_loss_hist.append(c_loss/n_batches)
    accuracy /= n_batches
    
    print("train_loss = {}, val_loss = {}, test_accuracy = {}".format(
        train_loss_hist[-1],
        test_loss_hist[-1],
        accuracy
    ))
    

### EPOCH 0 ###


/Users/davidlassner/Envs/wh4/lib/python3.7/site-packages/ipykernel_launcher.py:48: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.



train_loss = 0.4009570363653699, val_loss = 0.39143057058738395, test_accuracy = 0.8689159292035398


### EPOCH 1 ###




train_loss = 0.3749458155894684, val_loss = 0.3811656681705365, test_accuracy = 0.8689159292035398


### EPOCH 2 ###


KeyboardInterrupt: 

In [86]:
len(y_pred.numpy().ravel())
len(y_batch.numpy().ravel())

1 - sum(y_test)/len(y_test)

0.8688207677437172